In [149]:
import psycopg2
import boto3
from rdsiac import list_db 
from sql_queries import create_table_queries, drop_table_queries, song_table_insert, artist_table_insert, time_table_insert, user_table_insert
import glob
import os
import pandas as pd

In [144]:
rds_resource = boto3.client('rds')
list_db(rds_resource)

The MasterUsername is psadmin
The endpoint is mypostgresdb.cyudtagxfp4e.ap-southeast-2.rds.amazonaws.com, port 5432
The DB status is available
{'DBInstanceIdentifier': 'mypostgresdb', 'DBInstanceClass': 'db.t2.micro', 'Engine': 'postgres', 'DBInstanceStatus': 'available', 'MasterUsername': 'psadmin', 'Endpoint': {'Address': 'mypostgresdb.cyudtagxfp4e.ap-southeast-2.rds.amazonaws.com', 'Port': 5432, 'HostedZoneId': 'Z32T0VRHXEXS0V'}, 'AllocatedStorage': 20, 'InstanceCreateTime': datetime.datetime(2020, 6, 14, 4, 24, 39, 256000, tzinfo=tzutc()), 'PreferredBackupWindow': '15:05-15:35', 'BackupRetentionPeriod': 10, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-087a6b6e1b5e7e03a', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres11', 'ParameterApplyStatus': 'in-sync'}], 'AvailabilityZone': 'ap-southeast-2a', 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-c62829a1', 'Subnet

In [13]:
# connect to aws postgres
def connect_postgres():
    conn = psycopg2.connect("host = mypostgresdb.cyudtagxfp4e.ap-southeast-2.rds.amazonaws.com \
                             dbname = postgres \
                             user = psadmin \
                             password = "
                           )
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    cur.execute("DROP DATABASE IF EXISTS sparkifydb")
    cur.execute("CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0")
    conn.close()
    
    conn = psycopg2.connect("host = mypostgresdb.cyudtagxfp4e.ap-southeast-2.rds.amazonaws.com \
                             dbname = sparkifydb \
                             user = psadmin \
                             password = "
                           )
    cur = conn.cursor()
    return cur, conn

In [14]:
cur,conn = connect_postgres()

In [16]:
def drop_tables(cur,conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [17]:
# create tables for future process

In [18]:
drop_tables(cur,conn)

In [19]:
def create_tables(cur,conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [20]:
create_tables(cur,conn)

In [21]:
def check_tables(cur,conn):
    cur.execute("""SELECT table_name 
                   FROM information_schema.tables
                   WHERE table_schema = 'public'""")
    for table in cur.fetchall():
        print(table)

In [ ]:
# check if tables are created successfully

In [147]:
check_tables(cur,conn)

('artists',)
('songs',)
('time',)
('songplays',)
('users',)


In [ ]:
# next load the data on disk, process them

In [95]:
def get_files(filepath):
    datalist = []
    eventlist = []
    for file in glob.glob(os.path.join(filepath,'**/*.json'),recursive=True):
        if 'events' in file:
            eventlist.append(file)
        else:
            datalist.append(file)
    return datalist, eventlist


datalist, eventlist = get_files('./data/')

In [96]:
# insert into artist table

In [110]:
def insert_artist(datalist):
    count = 0
    for song in datalist:
        count += 1
        songdf = pd.read_json(song, lines=True)
        artist_data = songdf.loc[:,['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values
        cur.execute(artist_table_insert,list(artist_data[0]))
        conn.commit()
    print(f"total records inserted: {count}")

insert_artist(datalist)


total records inserted: 60


In [186]:
def check_results():
    #cur.execute("""rollback;""")
    cur.execute("""select * from users;""")
    print(cur.fetchall())
    conn.commit()

check_results()

[(90, 'Andrea', 'Butler', 'F', 'free'), (36, 'Matthew', 'Jones', 'M', 'paid'), (79, 'James', 'Martin', 'M', 'free'), (16, 'Rylan', 'George', 'M', 'free'), (42, 'Harper', 'Barrett', 'M', 'paid'), (2, 'Jizelle', 'Benjamin', 'F', 'free'), (26, 'Ryan', 'Smith', 'M', 'free'), (69, 'Anabelle', 'Simpson', 'F', 'free'), (63, 'Ayla', 'Johnson', 'F', 'free'), (48, 'Marina', 'Sutton', 'F', 'free'), (14, 'Theodore', 'Harris', 'M', 'free'), (86, 'Aiden', 'Hess', 'M', 'free'), (73, 'Jacob', 'Klein', 'M', 'paid'), (75, 'Joseph', 'Gutierrez', 'M', 'free'), (83, 'Stefany', 'White', 'F', 'free'), (62, 'Connar', 'Moreno', 'M', 'free'), (78, 'Chloe', 'Roth', 'F', 'free'), (50, 'Ava', 'Robinson', 'F', 'free'), (72, 'Hayden', 'Brock', 'F', 'paid'), (59, 'Lily', 'Cooper', 'F', 'free'), (8, 'Kaylee', 'Summers', 'F', 'free'), (80, 'Tegan', 'Levine', 'F', 'paid'), (29, 'Jacqueline', 'Lynch', 'F', 'paid'), (96, 'Cierra', 'Finley', 'F', 'free'), (15, 'Lily', 'Koch', 'F', 'paid'), (70, 'Jaleah', 'Hayes', 'F', 'pai

In [116]:
def insert_song(datalist):
    count = 0
    for song in datalist:
        count += 1
        songdf = pd.read_json(song, lines=True)
        song_data = songdf.loc[:,['song_id','title','artist_id','year','duration']].values
        cur.execute(song_table_insert,list(song_data[0]))
        conn.commit()
    print(f"total records inserted into song table: {count}")

insert_song(datalist)

total records inserted into song table: 60


In [ ]:
# part 2: event data

In [124]:
def read_events(eventlist):
    eventdf = pd.DataFrame()
    count = 0
    for event in eventlist:
        count += 1
        eachdf = pd.read_json(event, lines = True)
        print(f"read {eachdf.shape[0]} records from {count} file")
        eventdf = eventdf.append(eachdf)
    return eventdf

eventdf = read_events(eventlist)
eventdf.shape

read 95 records from 1 file
read 299 records from 2 file
read 157 records from 3 file
read 189 records from 4 file
read 15 records from 5 file
read 457 records from 6 file
read 223 records from 7 file
read 270 records from 8 file
read 59 records from 9 file
read 146 records from 10 file
read 171 records from 11 file
read 388 records from 12 file
read 202 records from 13 file
read 366 records from 14 file
read 379 records from 15 file
read 213 records from 16 file
read 283 records from 17 file
read 303 records from 18 file
read 533 records from 19 file
read 408 records from 20 file
read 327 records from 21 file
read 98 records from 22 file
read 96 records from 23 file
read 394 records from 24 file
read 517 records from 25 file
read 184 records from 26 file
read 433 records from 27 file
read 111 records from 28 file
read 382 records from 29 file
read 358 records from 30 file


(8056, 18)

In [175]:
def insert_time(eventdf):
    timecol = eventdf[eventdf['page'] == 'NextSong'].loc[:,'ts'].astype('datetime64[ms]')
    column_labels = ["timestamp", "hour", "day", "weelofyear", "month", "year", "weekday"]
    time_data = []
    for data in timecol:
        time_data.append([data ,data.hour, data.day, data.weekofyear, data.month, data.year, data.day_name()])
    time_df = pd.DataFrame.from_records(data = time_data, columns = column_labels)

    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))
        if i % 1000 == 0:
            print(f"finish insert {i} records")

insert_time(eventdf)


finish insert 0 records
finish insert 1000 records


KeyboardInterrupt: 

In [185]:
def insert_user(eventdf):
    userdf = eventdf[eventdf['page'] == 'NextSong'].loc[:,['userId','firstName','lastName','gender','level']]
    for i, row in userdf.iterrows():
        cur.execute(user_table_insert, list(row))
        if i % 1000 == 0:
            print(f"finish insert {i} records")

insert_user(eventdf)



finish insert 0 records
finish insert 0 records
finish insert 0 records
finish insert 0 records
finish insert 0 records
finish insert 0 records
finish insert 0 records


KeyboardInterrupt: 

In [187]:
cur.execute("""rollback;""")
conn.close()